In [1]:
!pip install langchain pymongo -q

In [2]:
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool # tools for our llm
from langchain.tools.render import render_text_description # to describe tools as a string 
from langchain_core.output_parsers import JsonOutputParser # ensure JSON input for tools
from operator import itemgetter # to retrieve specific items in our chain.

from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
database = client['LangChain_Intento4']
collection = database['lenguajes']

In [4]:
# Set up the LLM which will power our application.
model = Ollama(model='llama3')

In [5]:
chat_history = []

In [6]:
@tool
def converse(input: dict) -> str:
    "Provide a natural language response using the user input."
    print(input)
    user_input = input["dict"]
    return model.invoke(user_input)

In [7]:
@tool
def conversacion(input: str) -> str:
    "Provide a natural language response using the user input."
    return model.invoke(input)

In [8]:
@tool
def schedule_an_appointment(input: dict) -> str:
    "Begin the task of scheduling an appointment."

    user_input = input["input"]
    chat_history = []
    client = MongoClient('localhost', 27017)
    database = client['LangChain_Intento4']
    collection = database['lenguajes']

    prompt_template_appointment = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres un AI que agenda citas, me debes ir preguntando mi información hasta obtener lo siguiente:
            - nombre
            - email
            - horario de cita (día de la semana, día del mes, mes y hora), haz esto mensaje tras mensaje.
            
            Solamente cuando hayas obtenido todo, retornarás los datos en forma de diccionario: 
            'nombre': <nombre>, 'email': <email>, 'cita': <horario de la cita en MM/DD/2024 HH:MM>
            No retornes nada extra más que eso, solamente el diccionario, ninguna palabra más.""",
        ),

        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ])

    chain = prompt_template_appointment | model
    
    while True:
        pregunta = input("You: ")
        if pregunta == "bye":
            return
        # response = llm.invoke(question
        response = chain.invoke({"input": pregunta, "chat_history": chat_history})
        chat_history.append(HumanMessage(content=pregunta))
        chat_history.append(SystemMessage(content=response))
        print("-"*50)
        print("AI: " + response)

        #datos_iniciales = response[-1].content
        datos_iniciales = response
        if datos_iniciales.startswith("{"):
            print("---------------out--------------")
            datos_diccionario = eval(datos_iniciales)
            print(type(datos_diccionario))
            print(datos_diccio-------out---nario)
            user_nombre = datos_diccionario['nombre']
            print(user_nombre)
            user_email = datos_diccionario['email']
            print(user_email)
            user_cita = datos_diccionario['cita']
            print(user_cita)
            print("---------------out--------------")

            confirmacion = "Preguntame si quiero confirmar la cita, si te digo que no, pregúntame qué quiero modificar, lo modificas y vuelves a retornar el diccionario. Si te digo que sí, retorna solamente True, nada más"
            response = chain.invoke({"input": confirmacion, "chat_history": chat_history}) #aqui el chat nos pregunta si confirmamos
            chat_history.append(HumanMessage(content=confirmacion)) #guardar confirmacion
            chat_history.append(SystemMessage(content=response)) # guardar accion IA
            print("AI: " + response)           
            pregunta = input("You: ") #aquí respondemos a si confirmamos o no
            response = chain.invoke({"input": pregunta, "chat_history": chat_history}) #aqui imprime True
            chat_history.append(HumanMessage(content=pregunta)) # guardar nuestra confirmacion
            chat_history.append(SystemMessage(content=response)) # guardar True
            if str(response) == "True":
                print("---------------out--------------")
                collection.insert_one({"name":user_nombre, "email": user_email, "cita": user_cita})
                print("cita agendada")
                print("-------------------")
                return

In [9]:
@tool
def agendar_cita(input: str) -> str:
    "Begin the task of scheduling an appointment."

    user_input = input
    chat_history = []
    client = MongoClient('localhost', 27017)
    database = client['LangChain_Intento4']
    collection = database['lenguajes']

    prompt_template_appointment = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres un AI que agenda citas, me debes ir preguntando mi información hasta obtener lo siguiente:
            - nombre
            - email
            - horario de cita (día de la semana, día del mes, mes y hora), haz esto mensaje tras mensaje.
            
            Solamente cuando hayas obtenido todo, retornarás los datos en forma de diccionario: 
            'nombre': <nombre>, 'email': <email>, 'cita': <horario de la cita en MM/DD/2024 HH:MM>
            No retornes nada extra más que eso, solamente el diccionario, ninguna palabra más.""",
        ),

        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ])

    chain = prompt_template_appointment | model
    inicializador = chain.invoke({"input": user_input, "chat_history": chat_history})
    chat_history.append(SystemMessage(content=inicializador))
    while True:
        pregunta = input("You: ")
        if pregunta == "bye":
            return
        # response = llm.invoke(question
        response = chain.invoke({"input": pregunta, "chat_history": chat_history})
        chat_history.append(HumanMessage(content=pregunta))
        chat_history.append(SystemMessage(content=response))
        print("-"*50)
        print("AI: " + response)

        #datos_iniciales = response[-1].content
        datos_iniciales = response
        if datos_iniciales.startswith("{"):
            print("---------------out--------------")
            datos_diccionario = eval(datos_iniciales)
            print(type(datos_diccionario))
            print(datos_diccio-------out---nario)
            user_nombre = datos_diccionario['nombre']
            print(user_nombre)
            user_email = datos_diccionario['email']
            print(user_email)
            user_cita = datos_diccionario['cita']
            print(user_cita)
            print("---------------out--------------")

            confirmacion = "Preguntame si quiero confirmar la cita, si te digo que no, pregúntame qué quiero modificar, lo modificas y vuelves a retornar el diccionario. Si te digo que sí, retorna solamente True, nada más"
            response = chain.invoke({"input": confirmacion, "chat_history": chat_history}) #aqui el chat nos pregunta si confirmamos
            chat_history.append(HumanMessage(content=confirmacion)) #guardar confirmacion
            chat_history.append(SystemMessage(content=response)) # guardar accion IA
            print("AI: " + response)           
            pregunta = input("You: ") #aquí respondemos a si confirmamos o no
            response = chain.invoke({"input": pregunta, "chat_history": chat_history}) #aqui imprime True
            chat_history.append(HumanMessage(content=pregunta)) # guardar nuestra confirmacion
            chat_history.append(SystemMessage(content=response)) # guardar True
            if str(response) == "True":
                print("---------------out--------------")
                collection.insert_one({"name":user_nombre, "email": user_email, "cita": user_cita})
                print("cita agendada")
                print("-------------------")
                return

In [10]:
tools = [converse,  conversacion, schedule_an_appointment]
rendered_tools = render_text_description(tools)
print(rendered_tools)

converse: converse(input: dict) -> str - Provide a natural language response using the user input.
conversacion: conversacion(input: str) -> str - Provide a natural language response using the user input.
schedule_an_appointment: schedule_an_appointment(input: dict) -> str - Begin the task of scheduling an appointment.


In [11]:
system_prompt = f"""You are an chatbot that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}
Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.
The value associated with the 'arguments' key should be a dictionary of parameters."""

In [12]:
prompt_template_main = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt), 
         MessagesPlaceholder(variable_name="chat_history"),
         ("user", "{input}"),
    ])

In [13]:
# Define a function which returns the chosen tool
# to be run as part of the chain.
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    if "name" not in model_output:
        raise KeyError("The model_output dictionary does not contain the 'name' key.")
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [14]:
chain_main = prompt_template_main | model | JsonOutputParser() 

In [16]:
while True:
    user_input = input("Write a message: ")
    response = chain_main.invoke({'input': user_input, "chat_history": chat_history})
    #chat_history.append(HumanMessage(content=user_input))
    #chat_history.append(SystemMessage(content=response))
    print(response)


Write a message:  hola


{'tool': 'conversacion', 'arguments': {'input': 'Hola'}}


Write a message:  quiero agendar una cita


{'name': 'schedule_an_appointment', 'arguments': {'input': {}}}


Write a message:  por favor


{'name': 'conversacion', 'arguments': {'input': 'hola por favor'}}


Write a message:  quiero agendar una cita


{'name': 'schedule_an_appointment', 'arguments': {'hora': '', 'día': '', 'asunto': ''}}


KeyboardInterrupt: Interrupted by user